<a href="https://colab.research.google.com/github/adeshsingh5505/BotRush-2023/blob/main/webScrapping_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
excel_file = 'Input.xlsx'
df = pd.read_excel(excel_file)
urls = df['URL'].tolist()
ids = df['URL_ID'].tolist()
for url, url_id in zip(urls, ids):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Extracting title and body
    title = soup.title.text if soup.title else 'No title found'
    body2=soup.find(class_="td-post-content tagdiv-type")
    body=body2.get_text()


    content = f"Title: {title}\n\nBody:\n{body}"

    # saving content under URL_ID
    file_name = f"{url_id}.txt"
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(content)


In [6]:
# pushing evry file starting with bctech in folder
import os
import shutil

def move_bctech_files(src_dir, dest_dir):
    # Ensuring destination directory exists
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)


    for filename in os.listdir(src_dir):

        if filename.startswith('bctech') and filename.endswith('.txt'):
            # Constructing full file path
            src_file = os.path.join(src_dir, filename)
            dest_file = os.path.join(dest_dir, filename)

            # Moving the file
            shutil.move(src_file, dest_file)
            print(f"Moved: {filename}")

source_directory = '/content'  #source path
destination_directory = '/content/Analysis'  #  destination path

move_bctech_files(source_directory, destination_directory)


In [ ]:
pip install nltk

In [8]:
from zipfile import ZipFile
stopWords='/content/StopWords-20240711T223622Z-001.zip'
with ZipFile(stopWords,'r') as zip:
  zip.extractall()

In [ ]:

pip install chardet

In [ ]:
import os
import zipfile
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import defaultdict

# Downloading NLTK files
nltk.download('punkt')

def extract_zip(zip_file_path, extract_to_folder):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)

def load_stopwords(stopwords_folder):
    stop_words = set()
    for filename in os.listdir(stopwords_folder):
        if filename.endswith('.txt'):
            file_path = os.path.join(stopwords_folder, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                stop_words.update(word.strip().lower() for word in file.readlines())
    return stop_words

def clean_text(text, stop_words):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    cleaned_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    return cleaned_tokens

def create_sentiment_dict():
    # Define positive and negative words (example lists, you can expand these)
    positive_words = {'happy', 'good', 'great', 'excellent', 'positive', 'fortunate', 'correct', 'superior'}
    negative_words = {'sad', 'bad', 'terrible', 'awful', 'negative', 'unfortunate', 'wrong', 'inferior'}

    sentiment_dict = defaultdict(lambda: 'neutral')
    for word in positive_words:
        sentiment_dict[word] = 'positive'
    for word in negative_words:
        sentiment_dict[word] = 'negative'

    return sentiment_dict

def extract_derived_variables(tokens, sentiment_dict):
    positive_count = 0
    negative_count = 0

    for token in tokens:
        if sentiment_dict[token] == 'positive':
            positive_count += 1
        elif sentiment_dict[token] == 'negative':
            negative_count += 1

    return positive_count, negative_count

def read_file_with_multiple_encodings(file_path):
    encodings = ['utf-8', 'latin-1', 'windows-1252']  # List of encodings to try
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                return file.read()
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Unable to decode {file_path} with tried encodings")

def analyze_text_files(input_folder, output_excel_file, stopwords_folder):
    # Load stop words
    stop_words = load_stopwords(stopwords_folder)

    sentiment_dict = create_sentiment_dict()

    results = defaultdict(dict)

    # Iterate over all text files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):
            input_file_path = os.path.join(input_folder, filename)
            url_id = filename.rsplit('.', 1)[0]  # Extract url_id from filename (assumes filenames are like url_id.txt)

            # Reading stopwords
            try:
                text = read_file_with_multiple_encodings(input_file_path)
            except Exception as e:
                print(f"Error reading {filename}: {e}")
                continue

            cleaned_tokens = clean_text(text, stop_words)

            # Extract derived variables
            positive_count, negative_count = extract_derived_variables(cleaned_tokens, sentiment_dict)


            results['url_id'][url_id] = url_id
            results['Total Words'][url_id] = len(cleaned_tokens)
            results['Positive Words'][url_id] = positive_count
            results['Negative Words'][url_id] = negative_count


            print(f"Processed {filename}: Total Words={len(cleaned_tokens)}, Positive Words={positive_count}, Negative Words={negative_count}")


    df = pd.DataFrame(results)


    print("DataFrame Content:\n", df)

    # Saving to excel
    os.makedirs(os.path.dirname(output_excel_file), exist_ok=True)
    df.to_excel(output_excel_file, index=False)

    # Debug print to confirm saving
    print(f"Results saved to {output_excel_file}")

# Example usage
input_folder = '/content/Analysis'  # Replace with your input folder path
output_excel_file = '/content/Output Data Structure.xlsx'  # Replace with your output Excel file path
extracted_stopwords_folder = '/mnt/data/StopWords_extracted'  # Temporary folder for extracted stopwords

# Extract the uploaded zip file
zip_file_path = '/content/stopwords.zip'
extract_zip(zip_file_path, extracted_stopwords_folder)

# Run the analysis
analyze_text_files(input_folder, output_excel_file, extracted_stopwords_folder)


In [ ]:
pip install nltk textstat


In [28]:
import os
import re
import nltk
import pandas as pd
from nltk.corpus import cmudict
from textstat import textstat

# Ensure necessary resources are downloaded
nltk.download('punkt')
nltk.download('cmudict')

# Function to count syllables in a word
def count_syllables(word):
    d = cmudict.dict()
    if word.lower() in d:
        return len([y for y in d[word.lower()][0] if y[-1].isdigit()])
    else:
        return textstat.syllable_count(word)

# Function to calculate readability and other metrics
def analyze_text(text):
    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text)

    # Count personal pronouns
    personal_pronouns = re.findall(r'\b(I|we|my|ours|us)\b', text, re.I)

    # Calculate metrics
    num_sentences = len(sentences)
    num_words = len(words)
    num_complex_words = sum(1 for word in words if count_syllables(word) >= 3)
    total_syllables = sum(count_syllables(word) for word in words)
    avg_words_per_sentence = num_words / num_sentences if num_sentences > 0 else 0
    avg_syllables_per_word = total_syllables / num_words if num_words > 0 else 0
    avg_word_length = sum(len(word) for word in words) / num_words if num_words > 0 else 0

    # Analysis
    readability = textstat.flesch_reading_ease(text)

    return {
        "Readability Score": readability,
        "Average Number of Words Per Sentence": avg_words_per_sentence,
        "Complex Word Count": num_complex_words,
        "Word Count": num_words,
        "Syllable Count Per Word": avg_syllables_per_word,
        "Personal Pronouns": len(personal_pronouns),
        "Average Word Length": avg_word_length
    }

# Function to read text from a file and analyze it
def analyze_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    return analyze_text(text)

# Example usage
input_folder = '/content/Analysis'  # Replace with your input folder path
output_excel_file = '/content/Output Data Structure.xlsx'  # Replace with your output Excel file path

results = {}

for filename in os.listdir(input_folder):
    if filename.endswith('.txt'):
        url_id = filename.rsplit('.', 1)[0]  # Extract url_id from filename (assumes filenames are like url_id.txt)
        input_file_path = os.path.join(input_folder, filename)
        analysis_result = analyze_text_from_file(input_file_path)
        results[url_id] = analysis_result

# Convert results to DataFrame
df = pd.DataFrame(results).T

# Save to Excel
os.makedirs(os.path.dirname(output_excel_file), exist_ok=True)
df.to_excel(output_excel_file, index_label='url_id')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


KeyboardInterrupt: 